In [14]:
import json

projectName = "pronesting"
# Read JSON data from results file
with open(f"{projectName}_results.json", "r") as results_file:
    results = json.load(results_file)
    if isinstance(results, str):
        try:
            results = json.loads(results)
            print("results has been converted to:", type(results))
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

# Read JSON data from bars file
with open(f"{projectName}_bars.json", "r") as bars_file:
    bars = json.load(bars_file)

    if isinstance(bars, str):
        try:
            bars = json.loads(bars)
            print("Bars has been converted to:", type(bars))
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

    # Now `bars` should be a list if it was successfully converted
    print("Bars:", bars)
# Now `results` and `bars` are loaded as lists from the JSON files
print("Results:", results)
print("Bars:", type(bars))

results has been converted to: <class 'list'>
Bars has been converted to: <class 'list'>
Bars: [{'id': 'SN1', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9838.0, 'posNumber': 'c19'}], 'remaining_waste': 5162.0}, {'id': 'SN2', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9743.0, 'posNumber': 'c18'}], 'remaining_waste': 5257.0}, {'id': 'SN3', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9689.0, 'posNumber': 'c16'}], 'remaining_waste': 5311.0}, {'id': 'SN4', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9660.0, 'posNumber': 'c17'}], 'remaining_waste': 5340.0}, {'id': 'SN5', 'profile': 'ZZ260-2-25-67', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9300.0, 'posNumber': 'p3'}], 'remaining_waste': 5700.0}, {'id': 'SN6', 'profile': 'ZZ260-2-25-67', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9300.0, 'posNumber': 'p3'}], 'remaining_waste': 5700.0}, {'id': '

In [11]:
import pdfplumber
import io
import traceback
import openpyxl
import json
from jinja2 import Environment, FileSystemLoader
import pandas as pd
from datetime import datetime
from fastapi import UploadFile, HTTPException
from io import StringIO
import os
from collections import defaultdict
from openpyxl import load_workbook, Workbook, styles


template_dir = os.path.join("D:\\Python\\python-works\\html-pdf-template\\templates")
env = Environment(loader=FileSystemLoader(template_dir))


def calculate_cuts(df, stock_lengths):
    """Calculate cuts based on stock availability, track waste per profile, and track bars used."""
    stock_lengths = {key: [float(length) * 1000 for length in lengths]
                     for key, lengths in stock_lengths.items()}
    results = []
    bar_counter = 1  # Counter for assigning unique Stock profile POS
    bars = []  # List to store information about each bar

    # Flatten the dataframe to a list of all cuts
    cuts = []
    for _, row in df.iterrows():
        posNumber = row['posNumber']
        profile = row['profile']
        quantity = int(row['quantity'])
        length = row['length']
        if isinstance(length, str):
            # Extract the numeric part from a string
            length = float(length.split(" ")[0])
        else:
            # Already numeric, convert to float if necessary
            length = float(length)
        cuts.extend([(profile, length, posNumber) for _ in range(quantity)])

    # Sort cuts in descending order of length to make larger cuts first
    cuts = sorted(cuts, key=lambda x: x[1], reverse=True)

    

    # Process each cut
    for profile, length, posNumber in cuts:
        # Find a bar (either waste or new stock) to cut from
        cut_info, bar_id = find_best_stock_option(
            profile, length, posNumber, bars, stock_lengths)
        if cut_info:
            # Assign bar_id if new bar is created
            if cut_info['new_bar']:
                bar_counter += 1

            results.append({
                'posNumber': posNumber,
                'Profile': profile,
                'Cut Length': length,
                'Cut Info': cut_info['description'],
                'Waste After Cut': cut_info['remaining_waste'],
                'Stock profile POS': bar_id
            })
        else:
            print(f"No suitable stock length for cut {length}mm of profile {profile}")

    return results, bars


def find_best_stock_option(profile, length, posNumber, bars, stock_lengths):
    """Determine the best stock bar to cut from, considering existing waste and tracking individual bars."""
    sorted_lengths = sorted(stock_lengths.get(profile, []), reverse=True)

    # Try to find a waste piece from existing bars
    for bar in bars:
        if bar['profile'] == profile and bar['remaining_waste'] >= length:
            # Cut from existing bar
            bar['cuts'].append({'cut_length': length, 'posNumber': posNumber})
            bar['remaining_waste'] -= length
            return {
                'description': f"Cut from bar {bar['id']} (waste remaining {bar['remaining_waste']}mm)",
                'remaining_waste': bar['remaining_waste'],
                'new_bar': False
            }, bar['id']

    # If no waste piece is suitable, use a new stock bar
    for stock_length in sorted_lengths:
        if stock_length >= length:
            # Create a new bar
            bar_id = f"SN{len(bars) + 1}"
            new_bar = {
                'id': bar_id,
                'profile': profile,
                'stock_length': stock_length,
                'cuts': [{'cut_length': length, 'posNumber': posNumber}],
                'remaining_waste': stock_length - length
            }
            bars.append(new_bar)
            return {
                'description': f"Cut from new bar {bar_id} ({stock_length}mm)",
                'remaining_waste': new_bar['remaining_waste'],
                'new_bar': True
            }, bar_id

    # If no stock length is sufficient
    return None, None


def save_results_to_dataframe(results, bars, projectName):
    now = datetime.now()
    today_date = now.date()

    print([bar for bar in bars][:5])

    # Overview Data
    total_cuts = len(results)
    total_profiles = len(set([bar['profile'] for bar in bars]))
    total_length_m = sum([bar['stock_length']
                         for bar in bars]) / 1000  # Convert mm to m
    total_waste_m = sum([bar['remaining_waste']
                        for bar in bars]) / 1000  # Convert mm to m
    
    overview_template = env.get_template('overview_template.html.j2')
    overview_template_html = overview_template.render({
        "projectName": projectName,
        "dateGenerated":today_date,
        "totalProfiles":total_profiles,
        "totalLengthM": total_length_m,
        "totalWasteM":total_waste_m,
        "totalCuts":total_cuts
    })

    overview_data = [
        ["Overview and Summary", ""],
        ["Project Name", projectName],
        ["Date Generated", today_date],
        ["Total Profiles Type", total_profiles],
        ["Total Length (m)", total_length_m],
        ["Total Waste (m)", total_waste_m],
        ["Total Cuts", total_cuts],
    ]
    overview_df = pd.DataFrame(overview_data, columns=["", ""])

    material_agg = defaultdict(
        lambda: {'quantity': 0, 'total_length_m': 0.0, 'total_waste_mm': 0})

    for bar in bars:
        key = (bar['profile'], bar['stock_length'])
        material_agg[key]['quantity'] += 1
        # Convert mm to m
        material_agg[key]['total_length_m'] += bar['stock_length'] / 1000.0
        material_agg[key]['total_waste_mm'] += bar['remaining_waste']


    # Material Data
    material_data = []
    for (profile, stock_length), data in material_agg.items():
        # Calculate estimated waste percentage
        total_stock_length_mm = stock_length * data['quantity']
        waste_percentage = (
            data['total_waste_mm'] / total_stock_length_mm) * 100 if total_stock_length_mm else 0
        material_data.append([
            profile,
            stock_length,
            data['quantity'],
            round(data['total_length_m'], 2),
            round(waste_percentage, 2)
        ])

    cutlist_template = env.get_template('cutlist_template.html.j2')
    cutlist_template_html = cutlist_template.render(data = material_data)

    material_df = pd.DataFrame(material_data, columns=[
        "Profile Type", "Stock Length (mm)", "Quantity",
        "Total Length (m)", "Est Waste (%)"
    ])

    stock_profile_template_data = []

    # Cut List Data
    cut_data = []
    for bar in bars:
        entity = {
            "stockProfilePOS": bar['id'],
            "profileType": bar['profile'],
            "stockLength": bar['stock_length'],
            "wasteMM": bar['remaining_waste'],
            "cuts": []
        }
        # Empty row for separation
        cut_data.append(["Stock profile POS", bar['id']])
        # Empty row for separation
        cut_data.append(["Profile Type", bar['profile']])
        # Empty row for separation
        cut_data.append(["Stock Length", bar['stock_length']])
        cut_data.append(["Waste (mm)", bar['remaining_waste']])

        cut_data.append(["Pos Number", "Cut Lengths (mm)", "Quantity"])


        # Group cuts by 'posNumber' and 'cut_length'
        cut_counts = defaultdict(int)
        for cut in bar['cuts']:
            key = (cut['posNumber'], cut['cut_length'])
            cut_counts[key] += 1

        # Convert the grouped cuts into a list and sort for consistency
        grouped_cuts = [(posNumber, cut_length, quantity)
                        for (posNumber, cut_length), quantity in cut_counts.items()]
        # Sort by Pos Number and Cut Length
        grouped_cuts.sort(key=lambda x: (x[0], x[1]))

        # Add grouped cuts to cut_data
        for posNumber, cut_length, quantity in grouped_cuts:
            entity["cuts"].append({
                "posNumber": posNumber,
                "cutLength": cut_length,
                "quantity": quantity
            })
            cut_data.append([posNumber, cut_length, quantity])
        
        stock_profile_template_data.append(entity)
        cut_data.append([])  # Empty row for separation

    stock_profile_template = env.get_template('stock_profile_template.html.j2')
    stock_profile_html = stock_profile_template.render(data=stock_profile_template_data)

    cut_list_df = pd.DataFrame(cut_data)

    # Write to Excel
    try:
        with pd.ExcelWriter("Combined_Output.xlsx", engine="openpyxl") as writer:
            overview_df.to_excel(
                writer, sheet_name="Overview", index=False, header=False)
            material_df.to_excel(
                writer, sheet_name="Material List", index=False)
            cut_list_df.to_excel(
                writer, sheet_name="Cut List", index=False, header=False)

        # Merge Sheets
        pdf = merge_sheets_into_one("Combined_Output.xlsx", [
                              "Overview", "Material List", "Cut List"], overview_template_html, cutlist_template_html, stock_profile_html)
        return pdf
    except Exception as e:
        traceback.print_exc()


def merge_sheets_into_one(file_path, sheet_names, overview_html, cutlist_html, stock_profile_html ,output_sheet_name="MergedSheet"):
    # Load the workbook and select sheets to merge
    original_wb = load_workbook(file_path)
    new_wb = Workbook()
    merged_sheet = new_wb.active
    merged_sheet.title = "Data"  # Name for the merged sheet in the new workbook

    # Row counter to keep track of where to paste data in the merged sheet
    current_row = 1
    first_blank_row_encountered = False
    # Flag to ensure we start writing data only after valid rows begin
    has_written_data = False
    first_row_written = False  # Flag to indicate if the first row has been written
    # Border style
    bottom_border = styles.Border(bottom=styles.Side(
        border_style='thin', color='000000'))

    # Loop through each sheet specified in sheet_names
    for sheet_name in sheet_names:
        sheet = original_wb[sheet_name]
        if sheet_name == "Material List":
            merged_sheet.cell(row=current_row, column=1,
                              value="Material and Cut List")
            current_row += 2  # Move to the next row after adding the bold text

        elif sheet_name == "Cut List":
            current_row += 1
            merged_sheet.cell(row=current_row, column=1, value="Cut List and Optimization").font = styles.Font(
                bold=True, size=12)
            current_row += 2

        # Copy each row from the current sheet to the merged sheet
        for row in sheet.iter_rows(values_only=True):
            if not has_written_data and all(cell is None for cell in row):
                continue  # Skip to the next row
            has_written_data = True

            # Check if the current row is blank
            if all(cell is None for cell in row):
                # Leave the first encountered blank row empty and continue
                if not first_blank_row_encountered:
                    first_blank_row_encountered = True
                    current_row += 1  # Move to the next row in the merged sheet for insertion
                    continue

            # Copy each row from the current sheet to the merged sheet
            for col_num, cell_value in enumerate(row, start=1):
                cell = merged_sheet.cell(
                    row=current_row, column=col_num, value=cell_value)
            # Check if the cell in column A (first cell in the row) contains 'Waste'
            cell_in_column_a = row[0] if len(row) > 0 else None
            if cell_in_column_a and 'Waste' in str(cell_in_column_a):
                current_row += 1  # Insert a blank row by incrementing current_row
            current_row += 1  # Move to the next row in the merged sheet
            if not first_row_written:
                first_row_written = True
                current_row += 1

        current_row += 1

    # Apply bold formatting to the first column in the merged sheet
    for row in merged_sheet.iter_rows(min_row=1, max_row=current_row - 1, min_col=1, max_col=1):
        for cell in row:
            cell.font = styles.Font(bold=True)

    # Apply bold formatting to row 9 in the merged sheet
    for cell in merged_sheet[12]:  # Access all cells in row 9
        cell.font = styles.Font(bold=True)

    for row in range(1, current_row):  # Start from row 9 to the last row
        cell = merged_sheet.cell(row=row, column=2)  # Column 2 is "B"
        cell.alignment = styles.Alignment(horizontal="left")
    merged_sheet['A1'].font = styles.Font(bold=True, size=12)
    merged_sheet['A10'].font = styles.Font(bold=True, size=12)

    # Apply border and reset font starting from row 12 until the next blank row after row 12
    for row in range(13, current_row):
        # Check if the row is empty, if it is, stop applying styles
        if all(cell.value is None for cell in merged_sheet[row]):
            current_row += 1  # Move to the next row after the empty row
            break  # Stop processing further rows if a blank row is encountered

        for col_num in range(1, merged_sheet.max_column + 1):
            cell = merged_sheet.cell(row=row, column=col_num)
            # Apply the bottom border style to the cell
            cell.border = bottom_border
            # Reset the font to simple (non-bold)
            cell.font = styles.Font(bold=False)
    row_cell = merged_sheet.cell(row=1, column=1)
    # Iterate over cells in column A and make "Cut List and Optimization" bold with size 12
    row_tracker = 0
    for row in merged_sheet.iter_rows(min_row=1, max_row=current_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == "Cut List and Optimization":
                cell.font = styles.Font(bold=True, size=12)
                current_row += 1
                row_tracker = cell.row

            if cell.value == "Pos Number":
                for col_num in range(1, merged_sheet.max_column + 1):
                    row_cell = merged_sheet.cell(row=cell.row, column=col_num)
                    row_cell.font = styles.Font(bold=True)
            # Reset the font to simple (non-bold) after "Pos Number" row until the next "Pos Number" is detected
            if cell.value == "Pos Number":
                pos_number_row = cell.row
                for row in range(pos_number_row + 1, current_row):
                    next_cell = merged_sheet.cell(row=row, column=1)
                    if next_cell.value == "Stock profile POS":
                        break
                    for col_num in range(1, merged_sheet.max_column + 1):
                        row_cell = merged_sheet.cell(row=row, column=col_num)
                        row_cell.font = styles.Font(bold=False)

    # Apply bottom border to all cells with values after row_tracker
    for row in range(row_tracker + 1, current_row):
        for col_num in range(1, merged_sheet.max_column + 1):
            cell = merged_sheet.cell(row=row, column=col_num)
            if cell.value is not None:
                cell.border = bottom_border

    # Save the workbook (you can choose to overwrite or save as a new file)
    pdf = save_to_pdf(overview_html, cutlist_html, stock_profile_html)
    return pdf

from xhtml2pdf import pisa
def save_to_pdf(overview_html, cutlist_html, stock_profile_html):

    # Initialize HTML content
    html_content = f"<html><head></head><body>{overview_html+cutlist_html+stock_profile_html}</body></html>"
    
    pdf_buffer = io.BytesIO()
    pisa_status = pisa.CreatePDF(io.StringIO(html_content), dest=pdf_buffer)

    # Check for errors
    if pisa_status.err:
        print("Error in PDF creation:", pisa_status.err)
        return None
    
    pdf_buffer.seek(0)  # Rewind the buffer to the beginning for reading
    return pdf_buffer

In [41]:
results_pdf = save_results_to_dataframe(results,bars,projectName)
# Save the PDF buffer to a file
with open(f"{projectName}_results.pdf", "wb") as pdf_file:
    pdf_file.write(results_pdf.getbuffer())

[{'id': 'SN1', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9838.0, 'posNumber': 'c19'}], 'remaining_waste': 5162.0}, {'id': 'SN2', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9743.0, 'posNumber': 'c18'}], 'remaining_waste': 5257.0}, {'id': 'SN3', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9689.0, 'posNumber': 'c16'}], 'remaining_waste': 5311.0}, {'id': 'SN4', 'profile': 'SHS250*250*8.0', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9660.0, 'posNumber': 'c17'}], 'remaining_waste': 5340.0}, {'id': 'SN5', 'profile': 'ZZ260-2-25-67', 'stock_length': 15000.0, 'cuts': [{'cut_length': 9300.0, 'posNumber': 'p3'}], 'remaining_waste': 5700.0}]
